In [1]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.2", base_url="host.docker.internal")

In [2]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

from plum_chatbot.configs.settings import Settings

settings = Settings()

client = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="FAQ",
    embedding=embeddings,
)

/tmp/ipykernel_49044/340393869.py:8: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)


In [17]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    vector_store.as_retriever(search_type="mmr"),
    "retrieve_faq",
    "Search and return information about FAQs of an ERP system.",
)

In [18]:
retriever_tool.invoke("cosa devo mettere al campo Mail inoltro per conoscenza?")

'2. COME MODIFICA I DATI DEL PROFILO\nDalla pagina iniziale premere il pulsante Profilo\nLa procedura visualizza quindi la finestra nella quale l’amministratore può inserire\ne/o modificare i propri dati\n E’ importante terminare l’operazione di inserimento/modifica premendo il pulsante\nsalva.\n\n\n1. COME CONFIGUARE LE EMAIL E LA PEC\nAll’interno del portale è possibile configurare tutti gli indirizzi email e Pec dello\nStudio  e  decidere,  a  seconda  delle  esigenze,  da  quale  indirizzo  inviare  la\ncorrispondenza.  \nDal menu principale cliccare su Configurazioni \uf0e0 Parametri Email \nPremere ora il simbolo \n  per inserire i parametri dell’email \nCompilare i campi della finestra :\n- Descrizione  (inserire  il  riferimento  dell’email,  es:  Mario  oppure  Solleciti\noppure amministrazione, ecc.)\n- Tipo  (inserire per le email ‘ORDINARIA’ e per la pec ‘PEC’)\n- Indirizzo (riportare  l’indirizzo  dell’email  o  della  pec,  es:\nammistrazionerossi@gmail.com oppure info@st

In [21]:
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model

response_model = init_chat_model(
    "llama3.2", model_provider="ollama", base_url="host.docker.internal", temperature=0.0
)


def generate_query_or_respond(state: MessagesState):
    """Call the model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """
    response = response_model.bind_tools([retriever_tool]).invoke(state["messages"])
    return {"messages": [response]}

In [22]:
input = {"messages": [{"role": "user", "content": "a"}]}
generate_query_or_respond(input)["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  retrieve_faq (86506d8e-081d-4de6-b2c5-a1494d340295)
 Call ID: 86506d8e-081d-4de6-b2c5-a1494d340295
  Args:
    query: a
